In [1]:
import pandas as pd
import datetime
import vk_api
import os
import requests
import json
import random

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import sys

In [2]:
token = '4e6e771d37dbcbcfcc3b53d291a274d3ae21560a2e81f058a7c177aff044b5141941e89aff1fead50be4f'

In [3]:
vk_session = vk_api.VkApi(token=token)
vk = vk_session.get_api()

In [4]:
vk.messages.send(
    chat_id=1,
    random_id=2,
    message='Matrix has you ...')

0

In [5]:
df = pd.read_csv('/home/jupyter-an.karpov/shared/ads_data.csv.zip', compression='zip')

In [6]:
ad_data = df.groupby(['ad_id', 'event'], as_index=False) \
    .agg({'user_id': 'count'})

In [7]:
ad_data = ad_data.pivot(index='ad_id', columns='event', values='user_id').reset_index()

In [8]:
ad_data = ad_data.fillna(0).assign(ctr=ad_data.click / ad_data.view)

In [9]:
top_ctr = ad_data.query('click > 20 & view > 100').sort_values('ctr').tail(10)

In [10]:
top_ctr.to_csv('top_ctr_data.csv', index=False)

In [11]:
path = '/home/jupyter-an.karpov/lesson_7/top_ctr_data.csv'
file_name = 'top_ctr_data.csv'

path_to_file = path
upload_url = vk.docs.getMessagesUploadServer(peer_id=2000000001)["upload_url"]
file = {'file': (file_name, open(path_to_file, 'rb'))}

response = requests.post(upload_url, files=file)

json_data = json.loads(response.text)
json_data

{'file': '591438330|0|-1|856324|18a983d0b7|csv|403|top_ctr_data.csv|c2f9e346fc49058f43500d24dbf7ae44|844363341001fa9065f1f71631e3878c||||eyJkaXNrIjoiNyIsInBlZXJfc2VuZGVyIjoiLTE5NjA5OTUyOCJ9'}

In [12]:
saved_file = vk.docs.save(file=json_data['file'], title=file_name)

In [13]:
saved_file

{'type': 'doc',
 'doc': {'id': 555175609,
  'owner_id': 591438330,
  'title': 'top_ctr_data.csv',
  'size': 403,
  'ext': 'csv',
  'date': 1592214057,
  'type': 1,
  'url': 'https://vk.com/doc591438330_555175609?hash=f5e049dfcd9237bef6&dl=FUYTSNRQHE4TKMRY:1592214057:fde24a49275516c989&api=1&no_preview=1'}}

In [14]:
attachment = 'doc{}_{}'.format(saved_file['doc']['owner_id'], saved_file['doc']['id'])

In [15]:
attachment

'doc591438330_555175609'

In [16]:
vk.messages.send(
    chat_id=1,
    random_id=3,
    message='Топ объявлений по CTR', 
    attachment = attachment
)

0

In [48]:
import gspread
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials

In [59]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

my_mail = 'anatoly1804@gmail.com'

# Authorization
credentials = ServiceAccountCredentials.from_json_keyfile_name('heroic-venture-268009-1dfbcc34e5fa.json', scope)
gs = gspread.authorize(credentials)

In [60]:
# Name of the table in google sheets, 
# can be url for open_by_url
# or id (key) part for open_by_key
table_name = 'to_sequence'  # Your table
# Get this table
work_sheet = gs.open(table_name)

In [72]:
# Select 1st sheet
sheet1 = work_sheet.sheet1

# Get data in python lists format
data = sheet1.get_all_values()

In [73]:
data

[['price', 'group'],
 ['110', 'Group_1'],
 ['20', 'Group_2'],
 ['250', 'group_3'],
 ['500', 'group_4']]

In [74]:
headers = data.pop(0)

In [75]:
df = pd.DataFrame(data, columns=headers)

In [76]:
df

,price,group
0,110,Group_1
1,20,Group_2
2,250,group_3
3,500,group_4


In [67]:
df.sort_values('price', ascending=False)

,price,group
1,20,Group_2
0,110,Group_1


In [71]:
sheet1.append_row([500, 'group_4'])

{'spreadsheetId': '1gjyC_l8LMR3xq6h9w0GlbfsCpjl_V1j3dl9_0WxYyq8',
 'tableRange': "'Лист1'!A1:B4",
 'updates': {'spreadsheetId': '1gjyC_l8LMR3xq6h9w0GlbfsCpjl_V1j3dl9_0WxYyq8',
  'updatedRange': "'Лист1'!A5:B5",
  'updatedRows': 1,
  'updatedColumns': 2,
  'updatedCells': 2}}

In [77]:
# Looks like spreadsheet should be already present at the dist (so, run code in create table section)
spreadsheet_name = 'A new spreadsheet'
sheet = 'KarpovCorses2'
d2g.upload(df, spreadsheet_name, sheet, credentials=credentials, row_names=True)

<Worksheet 'KarpovCorses2' id:1323350636>

In [84]:
url = 'https://api-metrika.yandex.net/stat/v1/data?'
visits = 'metrics=ym:s:visits&dimensions=ym:s:date&id=44147844'

In [85]:
vistis_url = url + visits

In [86]:
vistis_request = requests.get(vistis_url)

In [87]:
vistis_request

<Response [200]>

In [88]:
json_data = json.loads(vistis_request.text)

In [90]:
json_data['data']

[{'dimensions': [{'name': '2020-06-05'}], 'metrics': [718.0]},
 {'dimensions': [{'name': '2020-06-02'}], 'metrics': [644.0]},
 {'dimensions': [{'name': '2020-06-03'}], 'metrics': [633.0]},
 {'dimensions': [{'name': '2020-06-04'}], 'metrics': [606.0]},
 {'dimensions': [{'name': '2020-06-01'}], 'metrics': [572.0]},
 {'dimensions': [{'name': '2020-06-06'}], 'metrics': [375.0]},
 {'dimensions': [{'name': '2020-06-07'}], 'metrics': [176.0]}]

In [95]:
y_df = pd.DataFrame([(i['dimensions'][0]['name'], 
  i['metrics'][0]) for i in json_data['data']], columns=['date', 
                                                        'visits'])

In [96]:
spreadsheet_name = 'A new spreadsheet'
sheet = 'Yandex_visits'
d2g.upload(y_df, spreadsheet_name, sheet, credentials=credentials, row_names=True)

<Worksheet 'Yandex_visits' id:197715693>

Дз и контест

In [2]:
!pip install vk_api

You should consider upgrading via the 'C:\Users\1\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [3]:
import pandas as pd
import datetime
import vk_api
import os
import requests
import json
import random

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import sys

In [21]:
token = 'vk1.a.VWsUsTsf9q9FY8tweQlpcNf4R4R2nETV4m7q3fVUtFrUZvfkEYdGetTC_w0Rl-tkOjEHN4aTtAFyWTjbk6bXj0RzQE9EcJjPJDfTWiG1Jq8RejCgPj3494YgcfnH1I9IabVtiyYNcvQCXGjC7luf2hWTF2KP2RGCpfNUksRok7rcdM8WWIP_1-KFL7miHxknX0iXj0YCYOaJb5sevc-kiQ'

In [34]:


my_id = 320826413 

vk_session = vk_api.VkApi(token=token)

vk = vk_session.get_api()

In [35]:
print(vk.messages.getConversations())

{'count': 1, 'items': [{'conversation': {'peer': {'id': 320826413, 'type': 'user', 'local_id': 320826413}, 'last_message_id': 2, 'in_read': 2, 'out_read': 2, 'last_conversation_message_id': 2, 'in_read_cmid': 2, 'out_read_cmid': 2, 'is_marked_unread': False, 'important': False, 'can_write': {'allowed': True}}, 'last_message': {'date': 1675673800, 'from_id': -218704903, 'id': 2, 'out': 1, 'attachments': [{'type': 'doc', 'doc': {'id': 651232194, 'owner_id': 320826413, 'title': '20221228_113013.jpg', 'size': 4398957, 'ext': 'jpg', 'date': 1675673800, 'type': 4, 'url': 'https://vk.com/doc320826413_651232194?hash=H9hh2ICQEKHJefai0yjU7ruyhFAGhpg5LzMsOP5nnQw&dl=FUZDCOBXGA2DSMBT:1675674603:AZzf1sm6lXQtWxzVR7U6ZvFXjzcOUC53mBYcTWdtUzz&api=1&no_preview=1', 'preview': {'photo': {'sizes': [{'src': 'https://sun9-55.userapi.com/c237131/u320826413/d3/-3/m_3aaca43d22.jpg', 'width': 130, 'height': 100, 'type': 'm'}, {'src': 'https://sun9-70.userapi.com/c237131/u320826413/d3/-3/s_3aaca43d22.jpg', 'width'

In [36]:
vk.messages.send(
    user_id=my_id,
    random_id=random.randint(1, 2 ** 31),
    message='Сообщение бота')

3

In [37]:
path_to_file = r'C:\Users\1\OneDrive\Изображения\Галерея Samsung\DCIM\Camera\20221228_113013.jpg'
file_name = '20221228_113013.jpg'

upload_url = vk.docs.getMessagesUploadServer(peer_id=my_id)["upload_url"]
file = {'file': (file_name, open(path_to_file, 'rb'))}

# Send request to post this doc on vk.com
response = requests.post(upload_url, files=file)

json_data = json.loads(response.text)

saved_file = vk.docs.save(file=json_data['file'], title=file_name)
attachment = 'doc{}_{}'.format(saved_file['doc']['owner_id'], saved_file['doc']['id'])

vk.messages.send(
    user_id=my_id,  # id of chat where to send
    random_id=random.randint(1, 2 ** 31),  # random number for message identification
    message='Панкейки!',  # message text, optional here
    attachment=attachment)  # attachment name

4

In [44]:
!pip install gspread
! pip install oauth2client
! pip install df2gspread

You should consider upgrading via the 'C:\Users\1\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\1\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


  Created wheel for df2gspread: filename=df2gspread-1.0.4-py3-none-any.whl size=11952 sha256=69594526c1bdf30202cb88d4d1ef5668c1d3a8566603f05223256b389457d307
  Stored in directory: c:\users\1\appdata\local\pip\cache\wheels\80\2f\29\b15c01e2e9fbfaae81e5f9e8a05735954fc125b7cd9f1423a5
Successfully built df2gspread


You should consider upgrading via the 'C:\Users\1\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [43]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials


# Specify path to your file with credentials
path_to_credential = 'C:/Users/1/Downloads/my-project-first-377009-c256a91990f3.json'

# Specify name of table in google sheets
table_name = 'Первый проект с API'

scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name(path_to_credential, scope)

gs = gspread.authorize(credentials)
work_sheet = gs.open(table_name)

# Select 1st sheet
sheet1 = work_sheet.sheet1

# Get data in python lists format
data = sheet1.get_all_values()

# Get header from data
headers = data.pop(0)

# Create df
df = pd.DataFrame(data, columns=headers)
df.head()

,0,456
0,1,1243
1,2,342


In [45]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']


my_mail = 'mihail.ru02@gmail.com'
path_to_credentials = 'C:/Users/1/Downloads/my-project-first-377009-c256a91990f3.json'


# Authorization
credentials = ServiceAccountCredentials.from_json_keyfile_name(path_to_credentials, scope)
gs = gspread.authorize(credentials)

#### Загрузка таблицы из гугл доков

# Name of the table in google sheets,
# can be url for open_by_url
# or id (key) part for open_by_key
table_name = 'Первый проект с API'  # Your table

# Get this table
work_sheet = gs.open(table_name)

# Select 1st sheet
sheet1 = work_sheet.sheet1

# Get data in python lists format
data = sheet1.get_all_values()

# Get header from data
headers = data.pop(0)

# Create df
df = pd.DataFrame(data, columns=headers)
df.head()

#### Создание своей таблицы

# Create empty table
table_name = 'A new spreadsheet'

sheet = gs.create(table_name)


# Make it visible to other guys
sheet.share(my_mail, perm_type='user', role='writer')

<Response [200]>

In [50]:
from df2gspread import df2gspread as d2g


# Create your df
df = pd.DataFrame({'name': ['Misha', 'Olesya', 'Kolya'], 'age': [20, 18, 30]})
# Looks like spreadsheet should be already present at the dist (so, run code in create table section)
sheet = 'Sheet1'
d2g.upload(df, table_name, sheet, credentials=credentials, row_names=True)

<Worksheet 'Sheet1' id:0>

Контест

Получение данных по ссылке на гугл таблицы

In [52]:
my_link = 'https://docs.google.com/spreadsheets/d/1e3PRXmMNXo9AZZnaol1aHgYxtH6KYaCDymEQlK6jaO8/export?format=csv'

In [53]:
from io import BytesIO
import requests


req = requests.get(my_link)
data = req.content

df = pd.read_csv(BytesIO(data))

In [54]:
df

,date,gender,users
0,2020-06-14,Not specified,150
1,2020-06-14,male,117
2,2020-06-14,female,21


In [62]:
round(df.query('gender == "male"').users * 100 / df.users.sum())

1    41.0
Name: users, dtype: float64

Файл с данными из Яндекс Метрики

In [64]:
df = pd.read_csv('C:/Users/1/JupyterProjects/Karpov_curses_lessons_and_projects/all_data/7_browser.csv')

In [65]:
df

,date,browser,user_type,users
0,2020-06-14,Google Chrome,People,139
1,2020-06-14,Chrome Mobile,People,35
2,2020-06-14,Google Chrome,Robots,28
3,2020-06-14,Yandex Browser,People,22
4,2020-06-14,Firefox,People,15
5,2020-06-14,Safari,People,13
6,2020-06-14,Firefox,Robots,8
7,2020-06-14,Android Browser,People,8
8,2020-06-14,Opera,People,5
9,2020-06-14,Mobile Safari,People,5


In [80]:
#вариант с np.where(предпочтительнее)
chrome_robots = pd.Series(np.where((df.user_type == 'Robots') & (df.browser.str.contains('Chrome')), 1, 0))

In [79]:
#вариант с генератором списков
[1 if user_type == 'Robots' and 'Chrome' in browser else 0 for browser, user_type in zip(df.browser, df.user_type)]

[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]

In [81]:
ad_sample = pd.read_csv('C:/Users/1/JupyterProjects/Karpov_curses_lessons_and_projects/all_data/sample_ads.csv')

In [82]:
ad_sample

,ad_id,user_id,click_type
0,12345,qwert,top
1,12345,fsssc,center
2,12345,vfvgg,NaN
3,12345,asrvv,NaN
4,12345,tynuy,top
5,12345,vreio,NaN
6,12345,oirbv,top
7,12345,kjvbe,NaN
8,12345,vkrej,center
9,12345,krjvb,top


In [84]:
ad_sample.click_type.notna().sum()

11

In [2]:
import requests
import json
from urllib.parse import urlencode

In [5]:
token = '6064030530:AAEK_HrkkNkb5QBEBw-tWHvqtZGvicx1Sqw'
api_pattern = f'https://api.telegram.org/bot{token}'
update = f'{api_pattern}/getUpdates'
info = requests.get(update)
info.status_code

200

In [6]:
info.json()

{'ok': True,
 'result': [{'update_id': 25848441,
   'message': {'message_id': 5,
    'from': {'id': 1701509038,
     'is_bot': False,
     'first_name': 'Михаил',
     'last_name': 'Куляскин',
     'username': 'alaska_bear',
     'language_code': 'ru'},
    'chat': {'id': 1701509038,
     'first_name': 'Михаил',
     'last_name': 'Куляскин',
     'username': 'alaska_bear',
     'type': 'private'},
    'date': 1677612860,
    'text': 'Привет салага'}}]}

In [7]:
chat_id = info.json()['result'][0]['message']['chat']['id']

In [9]:
send_message = f'{api_pattern}/sendMessage'

In [10]:
params = {'chat_id': chat_id, 'text': 'Приветики'}

In [12]:
response = requests.get(send_message, params=params)

In [13]:
response.text

'{"ok":true,"result":{"message_id":6,"from":{"id":6064030530,"is_bot":true,"first_name":"MyFirstTgBot","username":"My_1023_first_tg_bot"},"chat":{"id":1701509038,"first_name":"\\u041c\\u0438\\u0445\\u0430\\u0438\\u043b","last_name":"\\u041a\\u0443\\u043b\\u044f\\u0441\\u043a\\u0438\\u043d","username":"alaska_bear","type":"private"},"date":1677613167,"text":"\\u041f\\u0440\\u0438\\u0432\\u0435\\u0442\\u0438\\u043a\\u0438"}}'